# ML Pipeline Preparation

Follow the instructions below to help you create your ML pipeline.

### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [read_sql_table](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# download necessary NLTK data
import nltk
nltk.download(['punkt', 'wordnet'])


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/utilisareur/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/utilisareur/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
# import libraries
import re
import numpy as np
import pandas as pd
import pickle
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer


In [7]:
# load data from database
engine = create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql_table("Messages", engine)

X = df.message.values
Y = df.iloc[:,4:].values

### 2. Write a tokenization function to process your text data

In [4]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

In [5]:
def tokenize(text):
    # get list of all urls using regex
    detected_urls = re.findall(url_regex, text)
    
    # replace each url in text string with placeholder
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    # tokenize text
    tokens = word_tokenize(text)
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()

    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens


### 3. Build a machine learning pipeline
This machine pipeline should take in the message column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
# build pipeline
pipeline = Pipeline([
               ('vect', CountVectorizer(tokenizer=tokenize)),
               ('tfidf', TfidfTransformer()),
               ('clf', MultiOutputClassifier(RandomForestClassifier()))
           ])

    

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

# train classifier
#pipeline.fit(X_train, y_train)


In [8]:
#pipeline.get_params()

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's **classification_report** on each.

In [9]:
# predict on test data
#y_pred = pipeline.predict(X_test)

#report = classification_report(y_test, y_pred, output_dict=False)
#print(report)

    

### 6. Improve your model
Use grid search to find better parameters.

In [10]:

# specify parameters for grid search
parameters = {
    'vect__ngram_range': ((1, 1), (1, 2)),
    'vect__max_df': (0.5, 0.75, 1.0),
    'vect__max_features': (None, 5000, 10000),
    'tfidf__use_idf': (True, False),
    'clf__estimator__n_estimators': [50, 100, 200],
    'clf__estimator__min_samples_split': [2, 3, 4],
    
}

# create grid search object
cv = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.

Since this project focuses on code quality, process, and pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
cv.fit(X_train, y_train)


### 8. Try improving your model further. Here are a few ideas:
- try other machine learning algorithms
- add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
# save the model to disk
filename = 'finalized_model-cv.sav'
pickle.dump(cv, open(filename, 'wb'))

y_pred = cv.predict(X_test)

### 10. Use this notebook to complete train.py¶
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.